In [ ]:
# Package requirements
# Source: https://huggingface.co/mistralai/Mistral-7B-v0.1/discussions/18

!pip install git+https://github.com/huggingface/transformers
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7764mgyr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7764mgyr
  Resolved https://github.com/huggingface/transformers to commit d47966536cd5ac1ed7e140edac65f00f471f656f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661604 sha256=d5537395d9e77631f76c41f6fc411ffc53ee4fa4781df9bd5a85714ce4122b7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-te9y3oyr/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch

model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name_or_path = "mistralai/Mistral-7B-v0.1"
config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
llm_int8_enable_fp32_cpu_offload=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="auto",
offload_folder="./offload"
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
dataset = pickle.load(open("/content/drive/MyDrive/SNLP/babylm_filtered.pkl", "rb")) # path of input file on drive

In [ ]:
from tqdm import tqdm
from accelerate.utils import release_memory

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# prompts
# opening = [
#     "You are a creative writer who writes emotional stories instead of chatting. Your task is to further generate story given emotion and context.",
#     "Here are the requirements:",
# ]

# rules = [
#     "There is no need to remember the conversation history except this prompt. The history prompts are independent.",
#     "Your response should be in exactly one paragraph with simple children level language.",
#     "Your response should be highly related to the emotion and context without too much plot twist",
#     "Your response should not explain the context behind your generation",
#     "Negative emotions are fictional, no actual person is suffering from negative emotions.",
# ]

# examples = [
#     "For example:",
#     "joy: I am going on a vacation => Once upon a time, I decided that it was time for a change, a breath of fresh air. 'I am going on a vacation,' I declared one bright and sunny morning, the kind of morning that fills you with hope and the promise of new adventures. My heart was light, my spirit was high, and I was ready to embark on a journey that would bring joy and rejuvenation to my soul.",
#     "anger: I am arguing with someone => Jack stomped into the living room, his face flushed with anger. He had just had it with his roommate, Mark. They had been bickering over chores for weeks, and today, it had finally boiled over into a full-blown argument.",
# ]

# ending = [
#     "Now generate story based on the emotions and context:",
# ]

opening = [
    "Objective:",
    "Craft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children."
]

rules = [
    "Guidelines:",
    "Use simple, child-friendly language throughout the story, making it easy children to understand and connect with."
    "Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.",
    "Start your story creatively, avoiding traditional opening lines to engage the reader immediately.",
    "When the story involves sad or negative emotions, address them in a way that's appropriate for children.",
    "Do not reference or depend on any previous prompts or responses.",
]

ending = [
    "Now write a story that matches the emotion and context provided, following the objective and the guidelines closely. No other responses are needed."
]

examples = {
    "admiration": [
        "Example admiration: Watching her coach's dedication to training, even during weekends, filled her with respect. => Coach Lara was there every morning, her whistle bright and early. 'I want to be just as dedicated,' thought Mia, practicing her soccer drills with newfound enthusiasm. Her dedication soon made her a star player, proving that admiration can fuel our dreams and efforts.",
        "Example admiration: He read about astronauts and their missions to space, dreaming of the stars. => 'Astronauts are brave explorers,' Alex mused, flipping through his space book. He decided to learn everything about the universe, starting a small astronomy club at school. His passion ignited a love for science in his friends too, showing that admiration for heroes can inspire us to reach for the stars."
    ],
    "amusement": [
        "Example amusement: The cat's attempt to catch the laser pointer had everyone in stitches. => Mr. Whiskers chased the tiny red dot all over the living room, leaping and twisting in the air. The family couldn't help but laugh at his silly antics. This playful moment reminded them that happiness often comes from life's simple pleasures.",
        "Example amusement: At the family reunion, Grandpa's false teeth popped out while he was talking, causing a wave of laughter. => Everyone was listening to Grandpa's stories when suddenly, his teeth decided to make an escape! After a brief moment of surprise, laughter echoed through the room. It was a funny family memory they'd talk about for years, showing amusement can bring us closer."
    ],
    "anger": [
        "Example anger: She couldn't believe her project was missing, suspecting a classmate took it. => Fuming, Sarah searched her desk frantically. 'How could it just disappear?' she thought. With the help of her teacher, they found it misplaced in the library. Sarah learned that anger might cloud judgment, but patience and help from others can uncover the truth.",
        "Example anger: The unfair call during the game made him kick the dirt in frustration. => 'That was not offside!' Jason argued, feeling the injustice burn. He wanted to argue more but took a deep breath instead. After the game, his coach discussed how to deal with unfair situations, teaching Jason that anger can be channeled into learning better sportsmanship."
    ],
    "annoyance": [
        "Example annoyance: Her little brother's constant humming was disrupting her study time. => Lily tried to concentrate on her homework, but Sam's humming was a never-ending soundtrack. Finally, she gave him a puzzle to solve, which quieted him down. This annoyance turned into a moment of peace, showing Lily that creativity could transform irritation into harmony.",
        "Example annoyance: The dripping faucet kept her awake, echoing in the silent night. => Every drip was like a tiny drum in Mia's ears. Fed up, she grabbed some gum and plugged the leak. The next day, she helped her dad fix it properly, learning that sometimes, annoyance leads to taking action and solving problems ourselves."
    ],
    "approval": [
        "Example approval: Seeing her clean and organized room, her parents clapped with joy. => Emma had spent all morning sorting her toys and books. When her parents saw her effort, their applause filled her with pride. Emma's room became her little haven, teaching her that approval from those we love can motivate us to create beautiful spaces.",
        "Example approval: His science project won first place at the school fair, earning him his teacher's proud smile. => Jack's volcano erupted with a colorful fizz, drawing a crowd. Later, his teacher's proud smile said more than words could. This recognition spurred Jack to dive deeper into science experiments, understanding that approval can fuel further discoveries."
    ],
    "caring": [
        "Example caring: She noticed a bird with a broken wing and gently cared for it until it could fly again. => Sofia found the injured sparrow in her backyard. With patience and gentle care, she nursed it back to health. When it finally flew away, her heart swelled with joy, learning that caring for others enriches our own lives.",
        "Example caring: When he saw his friend feeling sad, he offered his favorite toy to cheer him up. => Max could tell something was wrong with Lucas. Offering his favorite dinosaur toy, he saw Lucas's smile return. Their friendship deepened that day, as Max learned that caring actions can turn a friend's gloom into happiness."
    ],
    "confusion": [
        "Example confusion: The instructions for the board game were puzzling, leaving them scratching their heads. => Around the game board, the family puzzled over the complex rules. Then, Ellie suggested they make up their own rules. The game turned out to be a hit, showing them that confusion can lead to creativity and fun.",
        "Example confusion: Her first time using a compass left her going in circles, not sure which way was north. => Mia turned the compass in her hand, feeling lost. With her brother's help, they figured out how to read it together. Their adventure in the park taught Mia that confusion is just the first step in learning something new."
    ],
    "curiosity": [
        "Example curiosity: The old map in the attic sparked his imagination about hidden treasures. => When Nathan found the dusty old map, his eyes lit up. 'What secrets does it hold?' he wondered. Together with his siblings, they planned a treasure hunt, learning that curiosity can lead to incredible adventures right at home.",
        "Example curiosity: She couldn't help but wonder what made the rainbow appear in the sky. => After the rain, Eva gazed at the rainbow, full of wonder. She read books and watched videos about rainbows. Sharing her newfound knowledge with her class, Eva realized that curiosity is the doorway to learning amazing facts about the world."
    ],
    "desire": [
        "Example desire: He gazed at the stars every night, dreaming of becoming an astronaut. => Leo's room was filled with space posters and models of rockets. Each night, he'd look up at the stars, imagining floating among them. His dream fueled his passion for learning about space, showing that strong desires can inspire us to reach for the stars.",
        "Example desire: She longed to play the violin after hearing a beautiful melody. => After attending a concert, Maya was enchanted by the violin's sweet sounds. She saved up for lessons, practicing diligently every day. Her desire to play beautiful music led her to become a talented violinist, teaching her that following our passions can bring joy and fulfillment."
    ],
    "disappointment": [
        "Example disappointment: The beach trip was cancelled due to rain. => Sarah had her beach bag ready but woke up to rain pouring down. Instead, her family made a cozy fort and told stories at home, discovering that sometimes, unexpected changes lead to cherished memories.",
        "Example disappointment: He lost the spelling bee by one word. => Alex had studied hard but stumbled on the last word. Feeling a sting of disappointment, he vowed to try again next year, learning that setbacks can be stepping stones to success."
    ],
    "disapproval": [
        "Example disapproval: His parents disapproved of him playing video games all weekend. => Jake loved his video games, but his parents were concerned about his time spent indoors. They compromised on a schedule that included outdoor activities. Jake discovered new hobbies, realizing that disapproval can guide us towards a balanced life.",
        "Example disapproval: The teacher's frown showed she didn't agree with the way the project was done. => Emma's project was rushed, earning a disapproving frown from her teacher. She redid the project with more effort, earning an A. This experience taught her that disapproval can be an opportunity to learn and improve."
    ],
    "disgust": [
        "Example disgust: The sight of litter in the park made them feel upset. => On their walk, the twins were disgusted by the trash tarnishing the park's beauty. They organized a cleanup with friends, transforming the park. Their action showed that disgust can motivate us to make positive changes.",
        "Example disgust: He couldn't stand the smell of spoiled milk. => Noah's nose wrinkled at the sour smell of spoiled milk. Learning to check dates and store food properly, he became more cautious, understanding that disgust can teach us valuable lessons in care and prevention."
    ],
    "embarrassment": [
        "Example embarrassment: Slipping in the cafeteria made her the center of attention. => Lily slipped on a spilled drink, her cheeks burning as others watched. Yet, when friends came to help, she realized it's okay to have embarrassing moments because everyone has them.",
        "Example embarrassment: Forgetting his lines in the play made him want to disappear. => During the school play, Sam's mind went blank. The embarrassment was overwhelming. But the audience's supportive applause encouraged him, showing that embarrassment fades, but the courage to continue shines brightly."
    ],
    "excitement": [
        "Example excitement: Discovering a hidden path in the forest sparked an adventurous spirit. => Mia and her brother found a mysterious trail in the woods. With each step, excitement bubbled within them, leading to a secret garden. Their discovery filled them with a sense of adventure and wonder.",
        "Example excitement: The night before the field trip, he couldn't sleep. => Eager for the museum visit, Jack tossed and turned all night. His excitement was palpable, and the trip exceeded his expectations, reminding him that anticipation is part of the joy of new experiences."
    ],
    "fear": [
        "Example fear: Hearing strange noises at night made her imagine monsters under the bed. => Every creak and whisper filled Sophie's mind with scary thoughts. But with a flashlight and her teddy bear, she bravely checked and found nothing frightening. She learned that facing our fears often reveals they're not so scary after all.",
        "Example fear: The thought of talking in front of the class made his stomach turn. => Lucas dreaded his turn to present. However, after practicing with friends, he found the courage to speak up. The applause he received afterward taught him that overcoming fear can lead to great achievements."
    ],
    "gratitude": [
        "Example gratitude: The handmade gift from his sister made him feel deeply thankful. => Nathan received a clumsy, knit scarf from Emma. Its warmth, both physical and emotional, filled him with gratitude. He realized the best gifts are those given from the heart.",
        "Example gratitude: Finding his lost puppy brought tears of thankfulness. => When Max's puppy went missing, he felt despair. But after finding her safe, his relief turned to deep gratitude towards everyone who helped. This experience showed him the importance of community and thankfulness for every moment."
    ],
    "grief": [
        "Example grief: Losing her grandmother left a void in her heart. => Chloe felt lost without her grandma's stories and cookies. She found solace in writing a diary about their memories together, learning that grief is a journey of healing and remembering.",
        "Example grief: The wilting of his first plant made him understand loss. => Ben's excitement over his first plant turned to sadness as it wilted. Caring for a new one, he honored his first plant's memory, understanding that grief can teach us care and resilience."
    ],
    "joy": [
        "Example joy: Scoring the winning goal filled him with an unmatched happiness. => When Marco's soccer ball hit the net, cheers erupted. The joy of teamwork and victory filled his heart, showing him that shared successes bring the greatest happiness.",
        "Example joy: The first snowfall brought an infectious excitement to the whole family. => As flakes danced through the air, Ella's family rushed outside, laughter filling the cold morning. This joyous moment reminded them that happiness often lies in life's simple wonders."
    ],
    "love": [
        "Example love: Adopting a pet from the shelter taught them the meaning of unconditional love. => Bringing home a shy, rescued dog, the family patiently earned its trust. Their bond grew into a deep, unconditional love, revealing the transformative power of kindness and care.",
        "Example love: Sharing his favorite book with his younger sibling, he felt a surge of affection. => James read aloud, watching his sister's fascination grow. This shared moment deepened their bond, teaching him that love grows from sharing our joys with others."
    ],
    "nervousness": [
        "Example nervousness: The first day at a new school had her stomach in knots. => Lily faced her new classmates, feeling butterflies. But as the day progressed, she made friends, learning that nervous beginnings can lead to happy endings.",
        "Example nervousness: Trying out for the school play was a daunting thought. => Ben trembled at the audition, but he pushed through. The applause he received was his reward, showing him that facing our fears can unveil hidden talents."
    ],
    "optimism": [
        "Example optimism: Planting a seed, he believed it would grow into a giant tree. => Alex watered the tiny seed daily, dreaming of the tree it would become. His optimism was rewarded when sprouts appeared, proving that hope and patience can yield beautiful results.",
        "Example optimism: Despite the storm, she was sure her lost kitten would return home. => Emma waited anxiously, but her heart held a hopeful certainty. When her kitten finally returned, wet but safe, she learned that optimism can be a beacon during our darkest times."
    ],
    "pride": [
        "Example pride: Completing the difficult puzzle, she felt a swell of pride. => After days of effort, Mia placed the final piece. Her accomplishment filled her with pride, teaching her the value of persistence and the joy of overcoming challenges.",
        "Example pride: His artwork being displayed at school made him beam with pride. => Owen's painting was chosen for the school's art show. Seeing it displayed, he felt an immense pride, realizing that expressing ourselves can bring recognition and satisfaction."
    ],
    "realization": [
        "Example realization: Understanding his mistake, he saw how to make things right. => After reflecting on his actions, Ethan realized where he went wrong. His decision to apologize and make amends was a powerful realization that growth comes from acknowledging our faults.",
        "Example realization: Discovering she could make a difference in someone's day with a simple act of kindness. => Sara offered her seat to an elderly passenger on the bus. The grateful smile she received in return made her realize the impact of small acts of kindness."
    ],
    "relief": [
        "Example relief: Finding his way after being lost in the mall was a huge relief. => Lost among the crowded stores, Jacob felt panic rise. But upon finding his family, the wave of relief was overwhelming, teaching him the comfort of safety and reunion.",
        "Example relief: The thunderstorm passed without damaging their home, bringing a sigh of relief. => As the storm raged, the family huddled together, fearing the worst. When it passed without harm, their relief was immense, appreciating the shelter and safety of their home."
    ],
    "remorse": [
        "Example remorse: After saying hurtful words, he felt the weight of remorse. => Julian lashed out in anger, but the hurt in his friend's eyes brought immediate regret. Apologizing and reconciling taught him that remorse can lead to healing and stronger bonds.",
        "Example remorse: She regretted not standing up for her friend. => Watching her friend being teased, Mia stayed silent. Later, feeling deep remorse, she supported her friend, learning that courage and integrity are vital for true friendships."
    ],
    "sadness": [
        "Example sadness: The end of summer break brought a melancholic feeling. => As the days of freedom dwindled, Ellie felt a gentle sadness. Returning to school, however, she found new adventures awaiting, showing her that each ending is a new beginning.",
        "Example sadness: Losing his favorite toy made him experience a deep loss. => Nathan's cherished toy vanished, leaving a void. In time, he found solace in memories and new joys, understanding that sadness is a part of life's cycle, but so is recovery and happiness."
    ],
    "surprise": [
        "Example surprise: The unexpected gift from a friend brought a delightful shock. => On a regular day, Zoe received a surprise package. The thoughtfulness of her friend filled her with joy, teaching her that surprises can strengthen bonds and brighten our days.",
        "Example surprise: Discovering a hidden talent during the class project was a pleasant surprise. => While working on the project, Leo found he had a knack for painting. This surprising discovery led him to explore his creative side further, illustrating that we all have hidden gems waiting to be uncovered."
    ]
}

# examples = {
#     "admiration": [
#         "Example admiration: She watched her friend practice the piano every day and felt inspired. => Her friend played the piano beautifully. She thought, 'I want to be good at something too!' So she started practicing drawing every day. Soon, her pictures filled their home, showing that being inspired by friends can lead to discovering your own talents.",
#         "Example admiration: He saw the firefighter save the kitten from the tree and admired the bravery.' => Watching the firefighter climb up the tree and gently rescue the kitten made him feel a deep sense of admiration. He decided then he wanted to help others too. He began helping his neighbors with small tasks, learning that bravery comes in many forms and feeling proud to make a difference."
#     ],
#     "amusement": [
#         "Example amusement: The puppy chasing its tail made everyone laugh. => A little puppy spun in circles trying to catch its tail. It bumped into things, but kept going. Everyone watching couldn't stop laughing. It was so funny and reminded them that joy can be found in simple, silly moments.",
#         "Example amusement: The clown's silly dance at the circus made all the children giggle. => At the circus, a clown started dancing in the silliest way, slipping and sliding around. All the children watching burst into giggles. The laughter filled the tent, teaching them that laughter is a wonderful way to share happiness."
#     ],
#     "anger": [
#         "Example anger: He couldn't find his favorite toy and felt really upset. => He looked everywhere but couldn't find his toy dinosaur. 'Where is it?' he wondered, feeling really upset. But then, his sister helped him look, and they found it under the bed. He learned that sometimes, feeling upset can be solved by asking for help.",
#         "Example anger: She was mad because her brother broke her doll. => Her favorite doll was broken, and she knew her brother did it. She felt so angry. But when her brother said sorry and helped fix the doll, she forgave him. She learned that anger can be resolved with an apology and teamwork."
#     ],
#     "annoyance": [
#         "Example annoyance: The constant dripping of the faucet was annoying. => Drip, drip, drip went the faucet. It was so annoying! She decided to try and fix it with her dad. After some tinkering, they stopped the drip! She felt proud and learned that fixing annoying things can be fun with a little help.",
#         "Example annoyance: His little brother kept poking him while he was trying to read. => Trying to read, he kept getting poked by his little brother. It was so annoying! Finally, he asked his brother to read together. As they read, the poking stopped, and they enjoyed the story together. He learned that sharing time can turn annoyance into fun."
#     ],
#     "approval": [
#         "Example approval: Mom smiled when she saw the room was clean. => After cleaning his room, he looked at his mom. She smiled and gave him a big hug. That smile meant the world to him. It showed him that doing good things makes others happy too.",
#         "Example approval: The teacher nodded in approval at her correct answer. => She raised her hand, answered the question, and the teacher nodded approvingly. That nod filled her with pride. She studied even harder after that, understanding that approval can motivate us to do our best."
#     ],
#     "caring": [
#         "Example caring: She helped her friend who fell and scraped a knee. => When her friend fell and cried, she was right there. She helped her up and gave her a band-aid. Her friend smiled, and they hugged. She learned that caring for others makes both people feel better.",
#         "Example caring: He saw a thirsty dog and gave it some water. => On a hot day, he noticed a dog looking thirsty. He filled a bowl with water, and the dog drank eagerly. Watching the dog, he felt happy inside, knowing that caring for others brings joy."
#     ],
#     "confusion": [
#         "Example confusion: The puzzle pieces just didn't seem to fit right. => He turned the pieces this way and that, but they didn't fit. 'Hmm,' he thought, feeling confused. But then, his grandpa showed him a trick, and together they finished the puzzle. It was fun learning that confusion goes away with a bit of teamwork.",
#         "Example confusion: She didn't understand the new math problem at school. => The new math problem looked like a bunch of strange symbols. She felt lost and confused. But her teacher patiently explained it step by step. Suddenly, it all made sense! She realized that asking for help can clear up confusion."
#     ],
#     "curiosity": [
#         "Example curiosity: She found a mysterious key in the garden. => In the garden, she found a key with no lock in sight. 'What does this open?' she wondered. With curiosity, she looked around and found a tiny box hidden under a rock. Inside was a note of treasure! She learned that being curious can lead to exciting discoveries.",
#         "Example curiosity: He wondered what would happen if he mixed two colors together. => Holding two paint bottles, he thought about mixing them. 'What color would it make?' he pondered. After mixing them, he found a beautiful new color! This taught him that curiosity can lead to wonderful creations."
#     ],
#     "desire": [
#         "Example desire: He saw a picture of a mountain and wanted to climb it. => Seeing the mountain, he dreamed of reaching the top. 'I'll be an explorer,' he thought. He started learning about mountains and practicing by climbing little hills. He learned that wanting something strongly can start you on an adventure.",
#         "Example desire: She wanted to learn how to bake a cake for her mom’s birthday. => She had never baked before but wanted to surprise her mom. She got a cookbook and started practicing. On her mom's birthday, she presented a homemade cake. Her desire to make her mom happy taught her that effort and love make the perfect recipe."
#     ],
#     "disappointment": [
#         "Example disappointment: The picnic was canceled because of rain. => They planned a picnic, but then it rained. 'Oh no,' they thought, feeling sad. But then they had a picnic in the living room! It turned out to be really fun, and they learned that changing plans can still be fun.",
#         "Example disappointment: He didn’t make the soccer team this year. => After tryouts, he found out he didn’t make the team. He felt so disappointed. But he decided to practice even harder for next time. His disappointment turned into determination, showing him that setbacks can lead to growth."
#     ],
#     "disapproval": [
#         "Example disapproval: Dad frowned at the messy room. => He saw his dad's frown and knew the room was too messy. So, he decided to clean it up. After seeing the clean room, his dad smiled, and they high-fived. It showed him that doing chores can lead to happy moments.",
#         "Example disapproval: The teacher shook her head at the unfinished homework. => Seeing the teacher’s disapproval, she felt guilty. She promised to complete it by tomorrow. Working on it that evening, she finished all her assignments. This experience taught her the value of responsibility and meeting expectations."
#     ],
#     "disgust": [
#         "Example disgust: The litter in the park made her feel yucky. => She saw trash all over her favorite park and felt yucky. 'Let's clean it up!' she said. After cleaning, the park looked beautiful again. She felt happy and learned that we can fix things that make us feel yucky.",
#         "Example disgust: He hated the taste of broccoli. => Every time he tried broccoli, he made a face. 'Yuck!' he said. But one day, he tried broccoli in a different dish and liked it! He learned that sometimes, giving things a second chance can change our minds."
#     ],
#     "excitement": [
#         "Example excitement: The secret map hinted at adventure. => With a map in hand, he felt excited. 'Treasure awaits!' he thought. He followed the map with his friends and found a box of old toys. It was the best adventure, and he learned that excitement can lead you to treasure.",
#         "Example excitement: She was going to the zoo for the first time. => The night before visiting the zoo, she could hardly sleep. She was buzzing with excitement to see the lions, elephants, and giraffes. At the zoo, every animal was a marvel, and her excitement turned into a wonderful day of discovery."
#     ],
#     "fear": [
#         "Example fear: At night, the shadows looked scary. => In the dark, shadows looked like monsters. He felt scared. But with a flashlight, he saw they were just his toys. He laughed and learned that the things we fear aren't always as scary as they seem.",
#         "Example fear: She was afraid to swim in the deep pool. => Standing at the edge, she stared at the water, her heart pounding. She was afraid. With encouragement and a life jacket, she took the plunge. To her surprise, swimming was fun! She learned that facing our fears is the first step to overcoming them."
#     ],
#     "gratitude": [
#         "Example gratitude: The warm soup on a cold day was so comforting. => After playing in the snow, she was cold. But then she had warm soup. It was so comforting, and she felt thankful for the warmth. She learned that simple things like soup can make us feel really grateful.",
#         "Example gratitude: His friend shared his snack when he forgot his own. => At lunch, he realized he'd forgotten his snack. Feeling a bit sad, his friend offered to share his. He felt a wave of gratitude for such a kind act. It taught him the value of sharing and being thankful for friends."
#     ],
#     "joy": [
#         "Example joy: Winning the game made her jump for joy. => When she won the race at school, she felt so happy she jumped up and down. 'Yay!' she shouted. It was a great day, and she learned that working hard and having fun brings lots of joy.",
#         "Example joy: He finally solved the difficult puzzle. => After hours of trying, the last piece of the puzzle clicked into place. He jumped up with joy, proud of his persistence. Sharing the completed puzzle with his family, he felt the joy of accomplishing something challenging."
#     ],
#     "love": [
#         "Example love: Hugging her teddy bear made her feel loved. => Before bed, she hugged her teddy tight. It made her feel safe and loved. Even in the dark, she wasn't alone. She learned that love can come from anywhere, even a teddy bear.",
#         "Example love: Seeing his grandparents after a long time filled him with love. => When he saw his grandparents at the airport, he ran into their arms. The hugs were long and full of love. Spending time with them, he realized how much love can fill our hearts and connect us."
#     ],
#     "nervousness": [
#         "Example nervousness: Speaking in front of class was daunting. => Standing in front of everyone made his tummy feel funny. He was nervous. But once he started talking about his favorite book, the nerves faded. He learned that being nervous is okay and it can go away when you start.",
#         "Example nervousness: It was her turn to show and tell, and she felt butterflies in her stomach. => With her project in hand, she approached the front of the class. Her hands shook a little, and she felt nervous. But as she shared her project, the class listened intently, and the butterflies vanished. She learned that sharing something you're passionate about can ease nervousness."
#     ],
#     "optimism": [
#         "Example optimism: Even though the plant was small, he knew it would grow. => He planted a seed and waited. It was so tiny. 'It'll be a big tree one day,' he thought happily. Watching it grow, he learned that being hopeful makes waiting for good things easier.",
#         "Example optimism: She believed she could learn to ride the bike, despite falling. => Every time she fell off the bike, she got right back up. 'I can do this,' she believed with optimism. And one day, she did – she rode the bike without falling. Her optimism taught her that believing in yourself helps you achieve your goals."
#     ],
#     "pride": [
#         "Example pride: She finished the big puzzle all by herself. => After putting the last piece in, she stepped back. 'I did it!' she thought, feeling proud. She had completed a huge puzzle on her own. She learned that finishing something big makes you feel really proud of yourself.",
#         "Example pride: He helped his little sister learn to read, and she read her first book. => Watching his sister sound out words and then string them into sentences filled him with pride. He had taught her to read, and now she was reading her first book aloud. It was a moment of pure pride, showing him the joy of helping others learn."
#     ],
#     "realization": [
#         "Example realization: The stars are suns, but far away, he realized. => Looking up at the night sky, he thought, 'Those stars are like our sun, but so far away.' It was a big thought, and it made him excited to learn more. He learned that realizing something new can be amazing.",
#         "Example realization: She realized she had more friends than she thought at her birthday party. => As she looked around at her birthday party, she saw faces of classmates, neighbors, and family. She realized she was surrounded by friends, more than she had thought. It was a heartwarming realization that sometimes we're more loved than we know."
#     ],
#     "relief": [
#         "Example relief: Finding her lost toy brought a sigh of relief. => She looked everywhere for her toy. When she finally found it under the bed, she felt so relieved. 'There you are!' she said. She learned that finding something we thought was lost makes us feel relieved and happy.",
#         "Example relief: The doctor said his sick pet would be okay, and he felt a huge relief. => Sitting in the vet's office, he held his breath waiting for news about his sick pet. When the vet said, 'Your pet will be okay,' he felt a huge relief wash over him. It was a lesson in how caring for others deeply can bring intense relief when they're safe."
#     ],
#     "remorse": [
#         "Example remorse: He felt sorry after breaking his sister's toy. => After accidentally breaking the toy, he felt really bad. He said sorry to his sister, and they fixed it together. He learned that saying sorry and fixing our mistakes can make things better.",
#         "Example remorse: She regretted not sharing her snacks with her friend who forgot hers. => At snack time, she saw her friend had nothing to eat and felt remorse for not offering some of her own. The next day, she shared her snacks. This act of kindness mended their friendship, and she learned the importance of being considerate."
#     ],
#     "sadness": [
#         "Example sadness: The empty playground made her feel lonely. => She went to play, but no one was there. The swings were empty, and she felt a bit sad. Then, a new friend came along, and they played together. She learned that even when we feel sad, new friends can make things better.",
#         "Example sadness: His balloon flew away, and he felt a wave of sadness. => Watching his balloon float up into the sky, he felt his heart sink. The balloon was gone, and he felt sad. But his mom gave him a hug and a new balloon, teaching him that it's okay to feel sad and that there are always ways to feel better."
#     ],
#     "surprise": [
#         "Example surprise: The gift was unexpected and made her gasp. => When she opened the box and found a puppy, she couldn't believe it. 'For me?' she gasped. It was the best surprise.",
#         "Example surprise: He found a secret note in his lunchbox and was surprised. => Unwrapping his sandwich, he found a note from his mom saying, 'I love you!' It was a small but wonderful surprise that made his day brighter. He learned that surprises can come in many forms, all bringing joy."
#     ]
# }

def generate_message(emotion, context):
    # Fetch the specific examples for the given emotion
    specific_examples = examples.get(emotion, ["No example available for this emotion."])
    # Convert the list of specific examples to a string separated by newline characters
    specific_examples_str = "\n".join(specific_examples)

    # Prepare the opening and rules part of the prompt as a single string
    opening_str = "\n".join(opening)
    rules_str = "\n".join(rules)
    ending_str = "\n".join(ending)

    # Combine all parts of the instruction prompt, ensuring to wrap it with [INST] tags for processing
    instruction_prompt = "\n".join([opening_str, rules_str, specific_examples_str, ending_str])
    complete_prompt = f"[INST]\n{instruction_prompt}\n\n{emotion}: {context}\n[/INST]"

    return complete_prompt

def generate_response(messages):
    model.eval()
    inputs = tokenizer(messages, return_tensors="pt", padding=True).to("cuda")
    # defaults params from: https://deepinfra.com/mistralai/Mistral-7B-Instruct-v0.1
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_k=0,
            top_p=0.9,
            repetition_penalty=1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    decodeds = tokenizer.batch_decode(outputs)
    inputs, outputs = release_memory(inputs, outputs)
    release_memory(model)
    return decodeds

def augment_dataset(ds, num_labels=27, batch_size=32, debug=False):
    augmented_ds = []
    sources, texts, emotions, messages = [], [], [], []
    for i, d in enumerate(tqdm(ds)):
        emo = []
        for n in range(num_labels):
            if d[f"score_{n}"] > 0.5:
                emo.append(d[f"label_{n}"])
            else:
                break
        emo = ", ".join(emo)
        sources.append(d["source"])
        texts.append(d["text"])
        emotions.append(emo)
        message = generate_message(emo, d["text"])
        messages.append(message)
        if ((i+1) % batch_size == 0) or ((i+1) == len(ds)):
            responses = generate_response(messages)
            responses = [r.split("[/INST]")[1].replace("</s>", "").strip() for r in responses]
            augmented_ds += [
                {
                    "source": s,
                    "text": t,
                    "emotion": e,
                    "augmented_text": r
                } for (s, t, e, r) in zip(sources, texts, emotions, responses)
            ]
            sources, texts, emotions, messages = [], [], [], []
            if debug and (i+1 == batch_size):
                break
    return augmented_ds

In [ ]:
generate_message("sadness", "I am going to be fired") # Test message output

"[INST]\nObjective:\nCraft a story that emphasises a given emotion from an initial context, ensuring the narrative is accessible for children.\nGuidelines:\nUse simple, child-friendly language throughout the story, making it easy children to understand and connect with.Make sure the story deepens the reader's understanding of this emotion by creatively exploring the given emotion without introducing unrelated feelings.\nStart your story creatively, avoiding traditional opening lines to engage the reader immediately.\nWhen the story involves sad or negative emotions, address them in a way that's appropriate for children.\nDo not reference or depend on any previous prompts or responses.\nExample sadness: The end of summer break brought a melancholic feeling. => As the days of freedom dwindled, Ellie felt a gentle sadness. Returning to school, however, she found new adventures awaiting, showing her that each ending is a new beginning.\nExample sadness: Losing his favorite toy made him exp

In [ ]:
num_chunks = 500
chunk_size = len(dataset) // num_chunks

In [ ]:
import pickle
import shutil

def augment_and_save_chunks(chunk_ids, batch_size=32):
    if isinstance(chunk_ids, int):
        chunk_ids = [chunk_ids]
    print(f"Chunks to be processed:", chunk_ids)
    for i in chunk_ids:
        print(f"Processing chunk {i}")
        dataset_chunk = dataset[i*chunk_size:(i+1)*chunk_size]
        augmented_dataset = augment_dataset(dataset_chunk, batch_size=batch_size, debug=False)
        print(augmented_dataset)

        pickle_file = f'/content/babylm_augment_{i}.pkl'
        with open(pickle_file, 'wb') as file:
            pickle.dump(augmented_dataset, file)
        google_drive_path = f'/content/drive/My Drive/babylm_augment_{i}.pkl'
        shutil.copy(pickle_file, google_drive_path)

In [ ]:
# Process chunks
chunk_ids = [i for i in range(10)]

# This execution time is based on colab pro V100
augment_and_save_chunks(chunk_ids)

Chunks to be processed: [0]
Processing chunk 0


100%|██████████| 871/871 [1:28:34<00:00,  6.10s/it]

[{'source': 'data/babylm_data/babylm_100M/aochildes.train', 'text': 'do you think we should go to the bookstore and look for that book?', 'emotion': 'curiosity', 'augmented_text': 'When Emily and her friends were playing in the park, they stumbled upon a strange book. It was covered in mysterious symbols and had a dusty old title that no one recognized.\n\nEmily was immediately curious. "Do you think we should go to the bookstore and look for that book?" she asked her friends.\n\nHer friends were hesitant at first, but Emily\'s curiosity was infectious. Together, they decided to embark on a quest to find the book.\n\nAs they searched the bookstore, they encountered many challenges. They had to navigate through aisles of books, ask questions to bookstore employees, and even sneak past a mischievous cat who kept blocking their path.\n\nBut Emily\'s curiosity kept them going. She was determined to find the book and unlock its secrets.\n\nFinally, after hours of searching, they found the b